# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('Orders/Orders.csv')
display(orders.shape)
orders.head()

(397924, 14)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here
orders_customer_spent = orders.groupby(['CustomerID']).agg({'amount_spent':sum})
orders_customer_spent.head(20)

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
12352,2506.04
12353,89.00
12354,1079.40
12355,459.40


In [4]:
orders_customer_spent['amount_spent'].describe()

count      4339.000000
mean       2053.793018
std        8988.248381
min           0.000000
25%         307.245000
50%         674.450000
75%        1661.640000
max      280206.020000
Name: amount_spent, dtype: float64

In [12]:
orders_new = orders_customer_spent[orders_customer_spent['amount_spent']>=1661.64]

In [13]:
orders_new.describe()

,amount_spent
count,1085.000000
mean,6499.120416
std,17216.740975
min,1661.840000
25%,2183.540000
50%,3075.040000
75%,5037.260000
max,280206.020000


### I have select the 75% quartile of total amount spent to define the class "Preferred".
### Then, I have looked at the top 25% values and select the 75% quartile value of the top 25% to define as "VIP". 

In [14]:
orders_customer_spent['Customer Level'] = pd.cut(orders_customer_spent['amount_spent'],[0,1661.64,5037.26,280207], labels=['Regular','Preferred','VIP'])
display(orders_customer_spent['Customer Level'].value_counts())
display(orders_customer_spent.head())

Regular      3253
Preferred     814
VIP           271
Name: Customer Level, dtype: int64

,amount_spent,Customer Level
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [19]:
# your code here
orders_2 = orders.groupby(['Country','CustomerID']).agg({'amount_spent':sum})
orders_2.head(20)

amount_spent
Country   CustomerID              
Australia 12386             401.90
          12388            2780.66
          12393            1582.60
          12415          124914.53
          12422             386.20
          12424            1760.96
          12431            5514.67
          12434             806.14
          16321             373.65
Austria   12358            1168.06
          12360            2662.06
          12370             277.20
          12373             364.60
          12374             742.93
          12414             562.41
          12429             437.98
          12453             707.09
          12817             166.04
          12818            1542.08
          12865            1568.23

In [20]:
def vip_check(x):
    if x>=5037.26:
        x = 1
    else: 
        x = 0
    return x

In [21]:
orders_2['VIP'] = orders_2['amount_spent'].apply(vip_check)
orders_2.head(20)

amount_spent  VIP
Country   CustomerID                   
Australia 12386             401.90    0
          12388            2780.66    0
          12393            1582.60    0
          12415          124914.53    1
          12422             386.20    0
          12424            1760.96    0
          12431            5514.67    1
          12434             806.14    0
          16321             373.65    0
Austria   12358            1168.06    0
          12360            2662.06    0
          12370             277.20    0
          12373             364.60    0
          12374             742.93    0
          12414             562.41    0
          12429             437.98    0
          12453             707.09    0
          12817             166.04    0
          12818            1542.08    0
          12865            1568.23    0

In [22]:
orders_3 = orders_2.groupby(['Country']).agg({'VIP':sum})
orders_3.sort_values(['VIP'],ascending=False)

,VIP
Country,
United Kingdom,226
Germany,12
France,10
Spain,3
Switzerland,3
Australia,2
EIRE,2
Portugal,2
Japan,2


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [23]:
orders_4 = orders.groupby(['Country','CustomerID']).agg({'amount_spent':sum})
def status_check(x):
    if x>=1661.64:
        x = 1
    else: 
        x = 0
    return x
orders_4['VIP'] = orders_4['amount_spent'].apply(status_check)
display(orders_4.head(20))
orders_5 = orders_4.groupby(['Country']).agg({'VIP':sum})
orders_5.sort_values(['VIP'],ascending=False)

amount_spent  VIP
Country   CustomerID                   
Australia 12386             401.90    0
          12388            2780.66    1
          12393            1582.60    0
          12415          124914.53    1
          12422             386.20    0
          12424            1760.96    1
          12431            5514.67    1
          12434             806.14    0
          16321             373.65    0
Austria   12358            1168.06    0
          12360            2662.06    1
          12370             277.20    0
          12373             364.60    0
          12374             742.93    0
          12414             562.41    0
          12429             437.98    0
          12453             707.09    0
          12817             166.04    0
          12818            1542.08    0
          12865            1568.23    0

,VIP
Country,
United Kingdom,932
Germany,39
France,29
Belgium,11
Switzerland,9
Spain,7
Portugal,7
Norway,7
Italy,5
